# Pong-ML: Offline & Online Model Training

This notebook trains and evaluates three ML models for Pong:
- **DT** (Decision Tree) - Offline only
- **HT** (Hoeffding Tree) - Offline + Online
- **WF** (Weighted Forest) - Offline + Online

---

## 1. Setup

In [ ]:
from src.data.setup import download_and_extract

download_and_extract(
    url="https://drive.google.com/uc?id=1STHDs5uR-bx-6beC36qtGaW_UqC1fY7s",
    zip_path="data.zip",
    extract_dir="data",
    verbose=True,
    remove_zip=True
)

---
## 2. Offline Training

Train all three models on tabular game data (humanvhuman + humanvpc).

In [ ]:
from src.models.decision_tree.train import main as train_dt
from src.models.hoeffding.train import main as train_ht
from src.models.weighted_forest.train import main as train_wf

train_dt()
train_ht()
train_wf()

---
## 3. Offline Evaluation (Tabular Data)

Repeated Stratified K-Fold Cross-Validation (10x5) with statistical tests.

In [ ]:
from src.evaluation.offline import run_offline_evaluation

CONFIG = {
    "n_repeats": 10,
    "n_splits": 5,
    "random_state": 42
}

results = run_offline_evaluation(**CONFIG)

---
## 4. Interactive Game (Optional)

Play against a trained model. Left paddle: Q (up), A (down). Right paddle: 

In [ ]:
from src.main import main as play_game

# Options: "human", "pc", "dt", "ht", "wf"
OPPONENT = "wf"

play_game(OPPONENT)

---
## 5. Online Training

Train HT and WF with reinforcement learning during simulated games.

In [1]:
from src.training.train_online import train_hoeffding_online, train_weighted_forest_online

# Configuration
ONLINE_CONFIG = {
    "num_episodes": 50,
    "max_score_per_episode": 5,
    "save_interval": 5
}

In [ ]:
train_hoeffding_online(
    pretrained_model_path="models/ht/hoeffding_tree_pong.pkl",
    **ONLINE_CONFIG
)

In [ ]:
train_weighted_forest_online(
    pretrained_model_path="models/wf/weighted_forest_pong.pkl",
    metadata_path="models/wf/weighted_forest_metadata.pkl",
    **ONLINE_CONFIG
)

---
## 6. Online Evaluation (Game Simulations)

Evaluate all models (pretrained + online) via simulated games against perfect AI.

In [ ]:
from src.evaluation.online import run_online_evaluation

# Configuration
EVAL_CONFIG = {
    "num_games": 20,
    "max_score": 5
}

results_df, stats = run_online_evaluation(**EVAL_CONFIG)

---
## 7. Visualization

In [ ]:
from src.visualization import plot_online_training_metrics

plot_online_training_metrics(
    ht_metrics_path="models/ht/hoeffding_online_metrics.csv",
    wf_metrics_path="models/wf/weighted_forest_online_metrics.csv"
)

---
## 8. Pretrained vs Online Comparison

In [7]:
from src.evaluation.online import compare_pretrained_vs_online

comparison = compare_pretrained_vs_online(max_score=5)